In [7]:
import kuzu
from pathlib import Path
import os


DATA_PATH = Path("../../../data/staging")
DB_NAME = "pecos_plus.db"
DEST_PATH = os.path.join(DATA_PATH, DB_NAME)

db = kuzu.Database(DEST_PATH)
conn = kuzu.Connection(db)

In [8]:
# Define the schema
results = conn.execute("CALL show_tables() RETURN *;")
while results.has_next():
    print(results.get_next())



['AffiliatedWith_LegalEntity_PECOSEnrolledCareProvider', 'REL', 'local(kuzu)', '']
['AffiliatedWith', 'REL_GROUP', 'local(kuzu)', '']
['EmployedBy', 'REL', 'local(kuzu)', '']
['OwnedBy_PECOSEnrolledCareProvider_LegalEntity', 'REL', 'local(kuzu)', '']
['LocatedAt_Person_Address', 'REL', 'local(kuzu)', '']
['AffiliatedWith_Person_LegalEntity', 'REL', 'local(kuzu)', '']
['LocatedAt', 'REL_GROUP', 'local(kuzu)', '']
['LocatedAt_PECOSEnrolledCareProvider_Address', 'REL', 'local(kuzu)', '']
['Person', 'NODE', 'local(kuzu)', '']
['OwnedBy_PECOSEnrolledCareProvider_Person', 'REL', 'local(kuzu)', '']
['Address', 'NODE', 'local(kuzu)', '']
['LegalEntity', 'NODE', 'local(kuzu)', '']
['OwnedBy', 'REL_GROUP', 'local(kuzu)', '']
['AffiliatedWith_Person_PECOSEnrolledCareProvider', 'REL', 'local(kuzu)', '']
['PECOSEnrolledCareProvider', 'NODE', 'local(kuzu)', '']


In [4]:
results.has_next()

False

Test a REL GROUP CREATE statement

In [ ]:
results = conn.exedcute("MATCH ")

In [29]:
results = conn.execute("""
    MATCH (p:Person) RETURN * LIMIT 10;
""")
print(results.get_next())

results = conn.execute("""
    MATCH (p:PECOSEnrolledCareProvider) RETURN * LIMIT 10;
""")
print(results.get_next())

[{'_id': {'offset': 0, 'table': 0}, '_label': 'Person', 'associate_id': '0042100026', 'first_name': 'PETER', 'middle_name': 'K', 'last_name': 'FUNG'}]
[{'_id': {'offset': 12288, 'table': 2}, '_label': 'PECOSEnrolledCareProvider', 'associate_id': '0042301640', 'enrollment_id': 'O20070810000797', 'enrollment_state': 'CA', 'provider_type_code': '00-06', 'provider_type_text': 'PART A PROVIDER - HOME HEALTH AGENCY', 'npi': '1467643809', 'multiple_npi_flag': 'N', 'ccn': '059092', 'organization_name': 'DIRECT NURSING SERVICES, INC.', 'doing_business_as_name': None, 'incorporation_date': '2007-06-20', 'incorporation_state': 'CA', 'organization_type_structure': 'CORPORATION', 'organization_other_type_text': None, 'proprietary_nonprofit': 'P', 'practice_location_type': None, 'location_other_type_text': None}]


In [28]:
"""
[{'_id': {'offset': 0, 'table': 0}, '_label': 'Person', 'associate_id': '0042100026', 'first_name': 'PETER', 'middle_name': 'K', 'last_name': 'FUNG'}]
[{'_id': {'offset': 12288, 'table': 2}, '_label': 'PECOSEnrolledCareProvider', 'associate_id': '0042301640', 'enrollment_id': 'O20070810000797', 'enrollment_state': 'CA', 'provider_type_code': '00-06', 'provider_type_text': 'PART A PROVIDER - HOME HEALTH AGENCY', 'npi': '1467643809', 'multiple_npi_flag': 'N', 'ccn': '059092', 'organization_name': 'DIRECT NURSING SERVICES, INC.', 'doing_business_as_name': None, 'incorporation_date': '2007-06-20', 'incorporation_state': 'CA', 'organization_type_structure': 'CORPORATION', 'organization_other_type_text': None, 'proprietary_nonprofit': 'P', 'practice_location_type': None, 'location_other_type_text': None}]
"""

result = conn.execute("""
    MATCH (p1:PECOSEnrolledCareProvider), (p2:Person) 
    WHERE p1.associate_id = '0042301640' AND p2.associate_id = '0042100026' 
    CREATE (p1)-[:OwnedBy {percentage_ownership: '10%', role_text: 'Big Dawg'}]->(p2)
""")

results = conn.execute("""
    MATCH (p:PECOSEnrolledCareProvider)-[r:OwnedBy]->(p2:Person) RETURN r.role_text, p2.first_name LIMIT 10;
""")

print(results.get_next())

['Big Dawg', 'PETER']


In [9]:
result = conn.execute("""
    MATCH (p1:PECOSEnrolledCareProvider), (p2:Person) 
    WHERE p1.associate_id = '0042301640' AND p2.associate_id = '0042100026' 
    CREATE (p1)-[:OwnedBy {
        role_text: $role_text,
        percentage_ownership: $percentage_ownership
    }]->(p2);
""",
parameters={"role_text": "test", "percentage_ownership": "10%"})

results = conn.execute("""
    MATCH (p:PECOSEnrolledCareProvider)-[r:OwnedBy]->(p2:Person) RETURN r.role_text, r.percentage_ownership, p2.first_name LIMIT 10;
""")

print(results.get_next())

['test', None, 'PETER']


In [5]:
statement2 = """MATCH (p:PECOSEnrolledCareProvider), (o:Person) WHERE p.enrollment_id = $enrollment_id AND o.associate_id = $associate_id CREATE (p)-[:OwnedBy {role_code $role_code, role_text $role_text, title $title, association_date $association_date, percentage_ownership $percentage_ownership}]->(o);"""